In [16]:
# ── Temperature Lag & Rolling Mean Feature + “fold‐wise” CV Evaluation ──
import sys, os
import numpy as np
import pandas as pd

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.preprocessing import load_and_merge
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor

# 1) 원본 데이터 로드 (병합 + 기존 피처 포함)
df = load_and_merge()

# 2) CV 셋업
tss    = TimeSeriesSplit(n_splits=5)
lags   = [1, 3, 6, 12, 24]
rolls  = [3, 6]
rmses  = []

for fold, (tr_idx, va_idx) in enumerate(tss.split(df), 1):
    # 3) fold 별로 train/val 분리
    train = df.iloc[tr_idx].copy()
    val   = df.iloc[va_idx].copy()

    # 4) train 구간에서만 lag/rolling 피처 생성
    for lag in lags:
        train[f"temp_lag_{lag}h"] = train["기온(degC)_관측"].shift(lag)
    for w in rolls:
        train[f"temp_roll_{w}h"] = (
            train["기온(degC)_관측"].shift(1)
                 .rolling(window=w)
                 .mean()
        )
    # lag/rolling 때문에 생긴 NaN 제거
    feature_cols = [
        "일사량(w/m^2)_예측", "기온(degC)_예측", "hour", "month", "weekday"
    ] + [f"temp_lag_{lag}h" for lag in lags] \
      + [f"temp_roll_{w}h" for w in rolls]

    train.dropna(subset=feature_cols + ["기온(degC)_관측"], inplace=True)

    # 5) val 에도 “train 에서 계산된” lag/rolling 결과만 사용
    #    -> train 과 val 을 합쳐 동일한 방식으로 shift/rolling 한 뒤, val 인덱스만 뽑아냄
    combined = pd.concat([train, val])
    for lag in lags:
        combined[f"temp_lag_{lag}h"] = combined["기온(degC)_관측"].shift(lag)
    for w in rolls:
        combined[f"temp_roll_{w}h"] = (
            combined["기온(degC)_관측"].shift(1)
                    .rolling(window=w)
                    .mean()
        )
    val = combined.loc[val.index]
    val.dropna(subset=feature_cols + ["기온(degC)_관측"], inplace=True)

    # 6) 모델 학습 & 예측
    X_tr, y_tr = train[feature_cols], train["기온(degC)_관측"]
    X_va, y_va = val[feature_cols],   val["기온(degC)_관측"]

    mdl = LGBMRegressor(random_state=42)
    mdl.fit(X_tr, y_tr)
    preds = mdl.predict(X_va)

    rmse = mean_squared_error(y_va, preds) ** 0.5
    print(f" Fold {fold} RMSE: {rmse:.4f}")
    rmses.append(rmse)

print(f"\n[Temperature CV RMSE] 평균: {np.mean(rmses):.4f}")



 Fold 1 RMSE: 3.8037
 Fold 2 RMSE: 1.9852
 Fold 3 RMSE: 0.5678
 Fold 4 RMSE: 0.5434
 Fold 5 RMSE: 0.6507

[Temperature CV RMSE] 평균: 1.5102


In [19]:
import numpy as np
import pandas as pd
import optuna
from lightgbm import LGBMRegressor, early_stopping
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

# ── 0) 데이터 준비 ──
df_full = load_and_merge()  # preprocessing.py 에서 병합+기존 피처까지 적용된 DataFrame
target_col   = "기온(degC)_관측"
base_feats   = ["일사량(w/m^2)_예측", "기온(degC)_예측", "hour", "month", "weekday"]
lags         = [1, 3, 6, 12, 24]
rolls        = [3, 6]
feature_cols = base_feats + [f"temp_lag_{lag}h" for lag in lags] + [f"temp_roll_{w}h" for w in rolls]

# ── 1) Objective 정의 ──
def objective(trial):
    # 1-1) 하이퍼파라미터 공간
    params = {
        "num_leaves":          trial.suggest_int("num_leaves", 16, 128),
        "learning_rate":       trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        "n_estimators":        trial.suggest_int("n_estimators", 50, 500),
        "max_depth":           trial.suggest_int("max_depth", 3, 15),
        "min_child_samples":   trial.suggest_int("min_child_samples", 5, 50),
        "subsample":           trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree":    trial.suggest_float("colsample_bytree", 0.6, 1.0),
    }

    # 1-2) Fold‐wise CV with safe feature‐creation
    tss   = TimeSeriesSplit(n_splits=5)
    rmses = []

    for tr_idx, va_idx in tss.split(df_full):
        train = df_full.iloc[tr_idx].copy()
        val   = df_full.iloc[va_idx].copy()

        # (a) train 구간에서만 lag/rolling 생성
        for lag in lags:
            train[f"temp_lag_{lag}h"] = train[target_col].shift(lag)
        for w in rolls:
            train[f"temp_roll_{w}h"] = (
                train[target_col].shift(1)
                      .rolling(window=w)
                      .mean()
            )
        train.dropna(subset=feature_cols + [target_col], inplace=True)

        # (b) val 은 train 기준으로 계산된 값만 사용하도록 병합 후 슬라이스
        combined = pd.concat([train, val])
        for lag in lags:
            combined[f"temp_lag_{lag}h"] = combined[target_col].shift(lag)
        for w in rolls:
            combined[f"temp_roll_{w}h"] = (
                combined[target_col].shift(1)
                        .rolling(window=w)
                        .mean()
            )
        val = combined.loc[val.index]
        val.dropna(subset=feature_cols + [target_col], inplace=True)

        # (c) 학습·예측
        X_tr, y_tr = train[feature_cols], train[target_col]
        X_va, y_va = val[feature_cols],   val[target_col]

        mdl = LGBMRegressor(**params, random_state=42, verbose=-1)
        mdl.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            callbacks=[early_stopping(stopping_rounds=50)]
        )
        preds = mdl.predict(X_va, num_iteration=mdl.best_iteration_)
        rmses.append(mean_squared_error(y_va, preds) ** 0.5)

    return np.mean(rmses)


# ── 2) Study 생성 & 최적화 ──
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

print("Best params:", study.best_params)
print("Best CV RMSE:", study.best_value)


[I 2025-06-23 14:19:22,570] A new study created in memory with name: no-name-619b70ee-d9e1-450b-bf4f-d72268c78389


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[483]	valid_0's l2: 37.8464
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[483]	valid_0's l2: 34.2588
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[483]	valid_0's l2: 7.66421
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[483]	valid_0's l2: 11.7536
Training until validation scores don't improve for 50 rounds


[I 2025-06-23 14:19:23,418] Trial 0 finished with value: 4.345932357625081 and parameters: {'num_leaves': 22, 'learning_rate': 0.0028846055059148492, 'n_estimators': 483, 'max_depth': 14, 'min_child_samples': 8, 'subsample': 0.7003039632879077, 'colsample_bytree': 0.9571980110324257}. Best is trial 0 with value: 4.345932357625081.


Did not meet early stopping. Best iteration is:
[483]	valid_0's l2: 12.4456
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[151]	valid_0's l2: 13.141
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[144]	valid_0's l2: 4.07095
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[131]	valid_0's l2: 0.34055
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[110]	valid_0's l2: 0.294183
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[207]	valid_0's l2: 0.420351


[I 2025-06-23 14:19:24,068] Trial 1 finished with value: 1.483402889685318 and parameters: {'num_leaves': 70, 'learning_rate': 0.06513617219170378, 'n_estimators': 298, 'max_depth': 9, 'min_child_samples': 6, 'subsample': 0.797308279819851, 'colsample_bytree': 0.8582731389040554}. Best is trial 1 with value: 1.483402889685318.


...

Did not meet early stopping. Best iteration is:
[286]	valid_0's l2: 0.412211
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[347]	valid_0's l2: 12.471
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[341]	valid_0's l2: 3.97535
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[297]	valid_0's l2: 0.326805
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[341]	valid_0's l2: 0.288243
Training until validation scores don't improve for 50 rounds


[I 2025-06-23 14:20:54,665] Trial 98 finished with value: 1.455753946138072 and parameters: {'num_leaves': 57, 'learning_rate': 0.027984371479388967, 'n_estimators': 347, 'max_depth': 12, 'min_child_samples': 11, 'subsample': 0.7224379304010267, 'colsample_bytree': 0.9317695674517916}. Best is trial 82 with value: 1.409050181360311.


Did not meet early stopping. Best iteration is:
[345]	valid_0's l2: 0.415975
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[346]	valid_0's l2: 11.6802
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[331]	valid_0's l2: 3.84184
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[222]	valid_0's l2: 0.32864
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[292]	valid_0's l2: 0.291746
Training until validation scores don't improve for 50 rounds


[I 2025-06-23 14:20:55,945] Trial 99 finished with value: 1.4263549741286137 and parameters: {'num_leaves': 67, 'learning_rate': 0.03640441465414191, 'n_estimators': 373, 'max_depth': 12, 'min_child_samples': 9, 'subsample': 0.89653426920452, 'colsample_bytree': 0.9698455363042231}. Best is trial 82 with value: 1.409050181360311.


Did not meet early stopping. Best iteration is:
[344]	valid_0's l2: 0.410472
Best params: {'num_leaves': 100, 'learning_rate': 0.07346470799079811, 'n_estimators': 245, 'max_depth': 13, 'min_child_samples': 9, 'subsample': 0.8993516398154066, 'colsample_bytree': 0.9749340842524324}
Best CV RMSE: 1.409050181360311


In [ ]:
# ── 02_feature_engineering.ipynb 에 추가할 셀 ──

import optuna
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor, Pool
from sklearn.ensemble import RandomForestRegressor

# 이미 위에서 정의된:
# df_full, lags, rolls, feat_cols, target_col = 
#   load_and_merge(), [1,3,6,12,24], [3,6], 
#   [...], "기온(degC)_관측"
feat_cols = feature_cols

# 1) CatBoost 용 objective
def objective_cb(trial):
    params = {
        "learning_rate":       trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        "depth":               trial.suggest_int  ("depth",          4,   12),
        "l2_leaf_reg":         trial.suggest_float("l2_leaf_reg",    1e-3, 10.0, log=True),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "border_count":        trial.suggest_int  ("border_count",   32,  255),
        "iterations":          trial.suggest_int  ("iterations",     100, 1000),
        "random_state":        42,
        "verbose":             False
    }
    tss = TimeSeriesSplit(n_splits=5)
    rmses = []
    for tr_idx, va_idx in tss.split(df_full):
        train = df_full.iloc[tr_idx].copy()
        val   = df_full.iloc[va_idx].copy()

        # (a) train 구간에만 lag/rolling
        for lag in lags:
            train[f"temp_lag_{lag}h"] = train[target_col].shift(lag)
        for w in rolls:
            train[f"temp_roll_{w}h"] = (
                train[target_col].shift(1)
                            .rolling(window=w)
                            .mean()
            )
        train.dropna(subset=feat_cols+[target_col], inplace=True)

        # (b) val 은 train 기준으로 계산된 피처만
        combined = pd.concat([train, val])
        for lag in lags:
            combined[f"temp_lag_{lag}h"] = combined[target_col].shift(lag)
        for w in rolls:
            combined[f"temp_roll_{w}h"] = (
                combined[target_col].shift(1)
                            .rolling(window=w)
                            .mean()
            )
        val = combined.loc[val.index]
        val.dropna(subset=feat_cols+[target_col], inplace=True)

        # (c) 학습 & 예측
        X_tr, y_tr = train[feat_cols], train[target_col]
        X_va, y_va = val  [feat_cols], val  [target_col]

        mdl = CatBoostRegressor(**params, early_stopping_rounds=50)
        mdl.fit(Pool(X_tr, y_tr), eval_set=Pool(X_va, y_va))
        preds = mdl.predict(X_va)
        rmses.append(mean_squared_error(y_va, preds) ** 0.5)

    return np.mean(rmses)

study_cb = optuna.create_study(direction="minimize")
study_cb.optimize(objective_cb, n_trials=100)
print("▶ CatBoost best params:", study_cb.best_params)
print("▶ CatBoost best CV RMSE:", study_cb.best_value)


# 2) RandomForest 용 objective
def objective_rf(trial):
    params = {
        "n_estimators":       trial.suggest_int("n_estimators",       50,  500),
        "max_depth":          trial.suggest_int("max_depth",          5,   30),
        "min_samples_split":  trial.suggest_int("min_samples_split",  2,   20),
        "min_samples_leaf":   trial.suggest_int("min_samples_leaf",   1,   10),
        "max_features":       trial.suggest_categorical("max_features", ["sqrt","log2"]),
        "random_state":       42,
        "n_jobs":             -1
    }
    tss = TimeSeriesSplit(n_splits=5)
    rmses = []
    for tr_idx, va_idx in tss.split(df_full):
        train = df_full.iloc[tr_idx].copy()
        val   = df_full.iloc[va_idx].copy()

        # (a) train 구간에만 lag/rolling
        for lag in lags:
            train[f"temp_lag_{lag}h"] = train[target_col].shift(lag)
        for w in rolls:
            train[f"temp_roll_{w}h"] = (
                train[target_col].shift(1)
                            .rolling(window=w)
                            .mean()
            )
        train.dropna(subset=feat_cols+[target_col], inplace=True)

        # (b) val 은 train 기준으로 계산된 피처만
        combined = pd.concat([train, val])
        for lag in lags:
            combined[f"temp_lag_{lag}h"] = combined[target_col].shift(lag)
        for w in rolls:
            combined[f"temp_roll_{w}h"] = (
                combined[target_col].shift(1)
                            .rolling(window=w)
                            .mean()
            )
        val = combined.loc[val.index]
        val.dropna(subset=feat_cols+[target_col], inplace=True)

        # (c) 학습 & 예측
        X_tr, y_tr = train[feat_cols], train[target_col]
        X_va, y_va = val  [feat_cols], val  [target_col]

        mdl = RandomForestRegressor(**params)
        mdl.fit(X_tr, y_tr)
        preds = mdl.predict(X_va)
        rmses.append(mean_squared_error(y_va, preds) ** 0.5)

    return np.mean(rmses)

study_rf = optuna.create_study(direction="minimize")
study_rf.optimize(objective_rf, n_trials=100)
print("▶ RF best params:", study_rf.best_params)
print("▶ RF best CV RMSE:", study_rf.best_value)


In [2]:
# ── Pressure Lag & Rolling Feature + fold‐wise CV 평가 ──
import sys, os
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor

# 프로젝트 루트 경로 설정 (notebook → project_root/src 접근)
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.preprocessing import load_and_merge

# 1) 데이터 로드 (병합 + 기존 파생피처 포함)
df = load_and_merge()

# 2) CV 설정 및 피처 리스트
tss    = TimeSeriesSplit(n_splits=5)
lags   = [1, 3, 6, 12, 24]
rolls  = [3, 6, 12, 24]
target_col    = "대기압(mmHg)_관측"
base_features = [
    "일사량(w/m^2)_예측",
    "습도(%)_예측",
    "절대습도_예측",
    "기온(degC)_예측",
    "대기압(mmHg)_예측",
    "hour", "month", "weekday"
]

rmses = []
for fold, (tr_idx, va_idx) in enumerate(tss.split(df), 1):
    train = df.iloc[tr_idx].copy()
    val   = df.iloc[va_idx].copy()

    # (a) train 구간에서만 lag/rolling 생성
    for lag in lags:
        train[f"pressure_lag_{lag}h"] = train[target_col].shift(lag)
    for w in rolls:
        train[f"pressure_roll_{w}h"] = (
            train[target_col].shift(1)
                   .rolling(window=w)
                   .mean()
        )
    feat_cols = base_features + \
                [f"pressure_lag_{lag}h" for lag in lags] + \
                [f"pressure_roll_{w}h" for w in rolls]
    train.dropna(subset=feat_cols + [target_col], inplace=True)

    # (b) val 에도 train 기준 피처만 적용 (combined trick)
    combined = pd.concat([train, val])
    for lag in lags:
        combined[f"pressure_lag_{lag}h"] = combined[target_col].shift(lag)
    for w in rolls:
        combined[f"pressure_roll_{w}h"] = (
            combined[target_col].shift(1)
                       .rolling(window=w)
                       .mean()
        )
    val2 = combined.loc[val.index].copy()
    val2.dropna(subset=feat_cols + [target_col], inplace=True)

    # (c) 학습·예측
    X_tr, y_tr = train[feat_cols], train[target_col]
    X_va, y_va = val2[feat_cols],  val2[target_col]

    mdl = LGBMRegressor(random_state=42, verbose=-1)
    mdl.fit(X_tr, y_tr)
    preds = mdl.predict(X_va)

    rmse = mean_squared_error(y_va, preds) ** 0.5
    print(f"Fold {fold} RMSE: {rmse:.4f}")
    rmses.append(rmse)

print(f"\n[Pressure CV RMSE] 평균: {np.mean(rmses):.4f}")


Fold 1 RMSE: 1.3250
Fold 2 RMSE: 0.9251
Fold 3 RMSE: 0.2058
Fold 4 RMSE: 0.3671
Fold 5 RMSE: 0.2960

[Pressure CV RMSE] 평균: 0.6238


In [10]:
# ── Pressure Optuna 하이퍼파라미터 튜닝 & Full‐Train 재학습 ──
import sys, os
import numpy as np
import pandas as pd
import optuna
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor, early_stopping
from catboost import CatBoostRegressor, Pool
from sklearn.ensemble import RandomForestRegressor

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.preprocessing import load_and_merge

# 1) 전체 데이터 로드 + lag/rolling 한번만 생성
df_full = load_and_merge()
lags   = [1, 3, 6, 12, 24]
rolls  = [3, 6, 12, 24]
target_col    = "대기압(mmHg)_관측"
for lag in lags:
    df_full[f"pressure_lag_{lag}h"] = df_full[target_col].shift(lag)
for w in rolls:
    df_full[f"pressure_roll_{w}h"] = (
        df_full[target_col].shift(1)
               .rolling(window=w)
               .mean()
    )
# NaN 행 제거
feat_cols = [
    "일사량(w/m^2)_예측",
    "습도(%)_예측",
    "절대습도_예측",
    "기온(degC)_예측",
    "대기압(mmHg)_예측",
    "hour", "month", "weekday"
] + [f"pressure_lag_{lag}h" for lag in lags] \
  + [f"pressure_roll_{w}h" for w in rolls]
df_full.dropna(subset=feat_cols + [target_col], inplace=True)

X_full = df_full[feat_cols]
y_full = df_full[target_col]

def objective(trial):
    params = {
        "num_leaves":        trial.suggest_int("num_leaves", 16, 128),
        "learning_rate":     trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        "n_estimators":      trial.suggest_int("n_estimators", 200, 500),
        "max_depth":         trial.suggest_int("max_depth", 5, 30),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "subsample":         trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree":  trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "lambda_l1":         trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2":         trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0.0, 1.0),
    }

    tss = TimeSeriesSplit(n_splits=5)
    rmses = []
    for tr_idx, va_idx in tss.split(df_full):
        train = df_full.iloc[tr_idx].copy()
        val   = df_full.iloc[va_idx].copy()

        # train 구간 lag/rolling
        for lag in lags:
            train[f"pressure_lag_{lag}h"] = train[target_col].shift(lag)
        for w in rolls:
            train[f"pressure_roll_{w}h"] = (
                train[target_col].shift(1)
                       .rolling(window=w)
                       .mean()
            )
        train.dropna(subset=feat_cols + [target_col], inplace=True)

        # val 구간 combined trick
        combined = pd.concat([train, val])
        for lag in lags:
            combined[f"pressure_lag_{lag}h"] = combined[target_col].shift(lag)
        for w in rolls:
            combined[f"pressure_roll_{w}h"] = (
                combined[target_col].shift(1)
                       .rolling(window=w)
                       .mean()
            )
        val2 = combined.loc[val.index].copy()
        val2.dropna(subset=feat_cols + [target_col], inplace=True)

        X_tr, y_tr = train[feat_cols], train[target_col]
        X_va, y_va = val2[feat_cols], val2[target_col]

        model = LGBMRegressor(**params, random_state=42, verbose=-1)
        model.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            callbacks=[early_stopping(stopping_rounds=50)]
        )
        preds = model.predict(X_va)
        rmses.append(mean_squared_error(y_va, preds) ** 0.5)

    return np.mean(rmses)

# 2) Optuna 최적화
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=150)

print("▶▶▶ Best params:", study.best_params)
print("▶▶▶ Best CV RMSE:", study.best_value)

# 3) Full‐Train 재학습 & RMSE
final = LGBMRegressor(**study.best_params, random_state=42, verbose=-1)
final.fit(X_full, y_full)
print("▶▶▶ Full-Train RMSE:", mean_squared_error(y_full, final.predict(X_full)) ** 0.5)


[I 2025-06-24 01:16:53,958] A new study created in memory with name: no-name-bdfec386-a383-4374-8e98-236a28b57bba


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[425]	valid_0's l2: 16.2063
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[425]	valid_0's l2: 13.5345
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[425]	valid_0's l2: 2.28261
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[425]	valid_0's l2: 5.25238
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[425]	valid_0's l2: 4.65203


[I 2025-06-24 01:16:54,687] Trial 0 finished with value: 2.7328245160033777 and parameters: {'num_leaves': 16, 'learning_rate': 0.00261466532037708, 'n_estimators': 425, 'max_depth': 30, 'min_child_samples': 76, 'subsample': 0.9255745599823761, 'colsample_bytree': 0.7594143576083004, 'lambda_l1': 2.520385027446699e-07, 'lambda_l2': 0.002597693415807421, 'min_gain_to_split': 0.5554052829301592}. Best is trial 0 with value: 2.7328245160033777.


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[243]	valid_0's l2: 4.55456
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[243]	valid_0's l2: 1.23026
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[243]	valid_0's l2: 0.0420237
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[243]	valid_0's l2: 0.143371
Training until validation scores don't improve for 50 rounds


[I 2025-06-24 01:16:55,322] Trial 1 finished with value: 0.8286639495178758 and parameters: {'num_leaves': 37, 'learning_rate': 0.030853662187034763, 'n_estimators': 243, 'max_depth': 26, 'min_child_samples': 61, 'subsample': 0.506598342976627, 'colsample_bytree': 0.9038296791407417, 'lambda_l1': 2.928376225269948e-06, 'lambda_l2': 0.00029401325242180257, 'min_gain_to_split': 0.08183741389172094}. Best is trial 1 with value: 0.8286639495178758.


...

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[224]	valid_0's l2: 2.02318
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[248]	valid_0's l2: 0.801154
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[251]	valid_0's l2: 0.0440705
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[263]	valid_0's l2: 0.135734
Training until validation scores don't improve for 50 rounds


[I 2025-06-24 01:19:25,781] Trial 148 finished with value: 0.6401330738904905 and parameters: {'num_leaves': 112, 'learning_rate': 0.03018353734382496, 'n_estimators': 305, 'max_depth': 30, 'min_child_samples': 17, 'subsample': 0.8934892707412437, 'colsample_bytree': 0.8826009737713579, 'lambda_l1': 9.460642150066158e-06, 'lambda_l2': 0.04196242251510203, 'min_gain_to_split': 0.05477822971492829}. Best is trial 121 with value: 0.620941341022382.


Did not meet early stopping. Best iteration is:
[276]	valid_0's l2: 0.0929376
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[201]	valid_0's l2: 2.18178
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[222]	valid_0's l2: 0.634081
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[236]	valid_0's l2: 0.0432275
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[237]	valid_0's l2: 0.138806
Training until validation scores don't improve for 50 rounds


[I 2025-06-24 01:19:27,056] Trial 149 finished with value: 0.6326679968984823 and parameters: {'num_leaves': 107, 'learning_rate': 0.034185630073779845, 'n_estimators': 297, 'max_depth': 29, 'min_child_samples': 9, 'subsample': 0.5394269688177562, 'colsample_bytree': 0.8713515090172026, 'lambda_l1': 0.00012413497907226754, 'lambda_l2': 0.017720913230680597, 'min_gain_to_split': 0.037624772704660164}. Best is trial 121 with value: 0.620941341022382.


Early stopping, best iteration is:
[247]	valid_0's l2: 0.0957808
▶▶▶ Best params: {'num_leaves': 103, 'learning_rate': 0.07388163691069162, 'n_estimators': 252, 'max_depth': 27, 'min_child_samples': 19, 'subsample': 0.8710148509302867, 'colsample_bytree': 0.8933785480332813, 'lambda_l1': 1.725469744768315e-05, 'lambda_l2': 0.04905364910420104, 'min_gain_to_split': 0.09056695749382586}
▶▶▶ Best CV RMSE: 0.620941341022382
▶▶▶ Full-Train RMSE: 0.16919833777447746
